In [ ]:
%matplotlib inline
import os
import cv2
import json
import torch
import torchvision
from torch import nn
import pandas as pd
import numpy as np
from skimage import color, io
import matplotlib.pyplot as plt

from torchvision import transforms
from torchvision.datasets import MNIST

from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader, ConcatDataset

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [ ]:
##############################################################
# CLASS TO NORMALIZE OUR DATA
##############################################################

class Normalization(object):
    def __init__(self, mean, std):
        self.mean = mean.view(-1, 1, 1)
        self.std = std.view(-1, 1, 1)
    def __call__(self, sample):
        image, label1, label2, label3 = sample['image'],\
        sample['label_age'], sample['label_gender'], sample['label_race']
        
        return {'image': image,
                'label_age': label1,
                'label_gender': label2,
                'label_race': label3}

##############################################################
# CLASS FOR DATA HANDLING IN PYTORCH
##############################################################

class MyData(Dataset):
    def __init__(self, image_tensor, annotations_tensor, train=True, transform=None):
        
        #Leaving only image related columns
        features=image_tensor
        y = annotations_tensor
        device = torch.device('cuda')
     
        ##########################################################
        # Setting labels
        ##########################################################
       
        label_chest_circumference=annotations_tensor[:,0]
        label_height=annotations_tensor[:,1]
        label_inseam=annotations_tensor[:,2]
        label_left_arm_length=annotations_tensor[:,3]
        label_pelvis_circumference=annotations_tensor[:,4]
        label_right_arm_length=annotations_tensor[:,5]
        label_shoulder_width=annotations_tensor[:,6]
        label_waist_circumference=annotations_tensor[:,7]
        
        ##########################################################
        # splitting the data into train and validation set
        ##########################################################
        
        X_train, X_test, y_train, y_test= train_test_split(features, y, test_size=0.2)
        
        if train==True:
            self.x=X_train
            self.y=y_train
        else:
            self.x=X_test
            self.y=y_test
            
        #############################
        # TRANSFORMS
        #############################
        
        # normalize data, w.o. Bessel Correction -> n instead of n-1
        #std, mean = torch.std_mean(self.x.float(), unbiased=False)

        #if transform is None:
            #intrinsic_transform = torch.nn.Sequential(transforms.Normalize(mean, std))
        
            #Applying transformation
            #self.transform=intrinsic_transform
        #else:
            #Applying transformation
            #self.transform=transform
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        image=torch.tensor(self.x[idx,:]).float()
        labels=torch.tensor(self.y[idx,:]).float()
        
        #Applying transformation
        #if self.transform:
            #image=self.transform(image)
        return (image,labels)

In [ ]:
##############################################################
# LOAD TENSORS FROM FILES
##############################################################

device = torch.device('cuda')

annotations_tensor = torch.load('tensors/humans_monsters_annotation_tensor.pt', map_location=device)
plain_image_tensor = torch.load('tensors/humans_monsters_plain_image_tensor.pt', map_location=device)

print(annotations_tensor[0])
print(plain_image_tensor[0])
#rgb_tensor = torch.load('tensors/humans_monsters_rgb_image_tensor.pt', map_location=device)
#texture_tensor = torch.load('tensors/humans_monsters_texture_image_tensor.pt', map_location=device)

##############################################################
# CREATE DATA OBJECTS AND PREPARE THEM FOR THE CNN WITH
# THE DATA LOADER CLASS
##############################################################

plain_dataset_train = MyData(plain_image_tensor, annotations_tensor, train=True)
plain_dataset_test = MyData(plain_image_tensor, annotations_tensor, train=False)
                      
test_loader = DataLoader(plain_dataset_test, batch_size=100, num_workers=0)
train_loader = DataLoader(plain_dataset_train, batch_size=100, num_workers=0)
                       
print("Dataset/s loaded")

#### Original Architecture

* Input Layer: fixed image of size 200 x 200 x 1
* Second Layer: Convolution with 5-pixels square kernel. Output: feature map of size 196 x 196 x 8.

* RelU: The tensor is then passed through a ReLU and batch normalization is applied.

* Pooling Layer: Max pooling with stride 2 
* Convolution Layer: Convolution with 5-pixels square kernel and 16 output channels Output: Tensor of size 94 x 94 x 16.

* Pooling Layer: Max pooling with stride 2. And then flatten. Output: tensor of size 35344

* Fully Connected Layer:
* RelU: The tensor is then passed through a ReLU and batch normalization is applied.

* Regressor Layer: Output: 8 human body dimensions in meters

#### Training
* 20 epochs
* mini_batc_size = 100
* Loss = MSE
* learning_rate = 0.01
* momentum = 0.9
* TODO: regressor funktion
* For layers
* 
* https://pytorch.org/tutorials/recipes/recipes/defining_a_neural_network.html

In [ ]:
class ConvNet(nn.Module):
    '''
    Simple Convolutional Neural Network
    '''
    def __init__(self):
        
        super().__init__()
        self.layers = nn.Sequential(

          # input layer 200 x 200 x 1 - (in: 1 color channel, out: 8 Channels, ...)
          # output 196 x 196 x 8  
          nn.Conv2d(1, 8, kernel_size=5),
          nn.ReLU(),

          # input 196 x 196 x 8 
          # output 98 x 98 x 8
          nn.MaxPool2d(stride=2, kernel_size=1),

          # input 98 x 98 x 8
          # output 94 x 94 x 16
          nn.Conv2d(8, 16, kernel_size=5),
          nn.ReLU(),

          # input 94 x 94 x 16
          # output 47 x 47 x 16
          nn.MaxPool2d(stride=2, kernel_size=1),

          # Pooling Layer: Max pooling with stride 2. And then flatten.
          # Output: tensor of size 35344 = 47 x 47 x 16
          nn.Flatten(),
          nn.Linear(35344, 35344),
          nn.ReLU(),

          # Last Layer - Regressor
          # input 35344
          # output 8
          nn.Linear(35344, 8)

        )

    def forward(self, x):
        inputs = x.unsqueeze(1)
        outputs = self.layers(inputs)
        return outputs

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(0) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.MSELoss()
model = ConvNet().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(test_loader, model, loss_fn, optimizer)
    test_loop(train_loader, model, loss_fn)
print("Done!")